In [26]:
import pandas as pd
import json
import numpy as np
from openai import OpenAI
import anthropic
from dotenv import load_dotenv
from pathlib import Path
import base64
import httpx
import ollama

In [33]:
def get_text_description_for_row_chatgpt(df_row: pd.Series, model: str = "gpt-4o-mini"):

    try:

        category = df_row['Category']
        product_name = df_row['ProductName']
        brand = df_row['Brand']
        price = df_row['Price']
        text_description = df_row['TextDescription']
        bullet_points = df_row['BulletPointDescription']

        input_message = "Here is the information on the item.\n- Product Name: {product_name}\n- Category: {category}\n- Brand: {brand}\n- Price: {price}\n- Description: {description}\n- Bullet point information: {bullet_points}"

    # print(content)
    
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "You are acting as an assistant designed to give a description about an item from a website marketplace, make sure the descriptions is purely descriptive and is not trying to sell the item. Use the information I am providing to describe the item in detail. Begin by identifying the type and category, then specify the material, brand, main color, and any secondary colors or patterns. Describe the fit and style and include any notable design features and whether it is a luxury item or not. Add the typical occasion or style it's intended for, and any other unique characteristics."
                },
                {
                    "role": "user",
                    "content": input_message.format(product_name=product_name, category=category, brand=brand, price=price, description=text_description, bullet_points=bullet_points)
                }
            ]
        )

        return completion.choices[0].message.content
    
    except:
        print("Error with item: ", df_row["ProductName"])
        return "error"

In [34]:
df = pd.read_csv('ItemData.csv', index_col=0)

In [37]:
df['AIDescription'] = df.apply(lambda x: get_text_description_for_row_chatgpt(x), axis=1)

In [38]:
df.to_csv('ItemDataWithAiDescription.csv')

In [39]:
df

,ProductName,Brand,Price,Category,Sizes,TextDescription,BulletPointDescription,ImageUrls,AIDescription
0,1017 ALYX 9SM Circle Logo Popover Hoodie,1017-alyx-9sm,325.0,Hoodies & Sweats,[],Drawing influences from California skate cultu...,"['100% Cotton', 'Drawstring Hood', 'Kangaroo P...","['https://media.endclothing.com/media/f_auto,q...",The item is a hoodie from the category of Hood...
0,1017 ALYX 9SM Side Zip Camo Pant,1017-alyx-9sm,443.0,Trousers,[],1017 ALYX 9SM innovative designs take cues fro...,"['100% Nylon', 'Elasticated Waist', 'Zip Closu...","['https://media.endclothing.com/media/f_auto,q...",The item is categorized as trousers and is spe...
0,1017 ALYX 9SM x Blackmeans Denim Jacket,1017-alyx-9sm,779.0,Coats & Jackets,[],Working together on a capsule that creates a r...,"['100% Cotton', 'Classic Collar', 'Button Clos...","['https://media.endclothing.com/media/f_auto,q...",The item is a denim jacket categorized under C...
0,1017 ALYX 9SM Oversized Camo Nylon Bomber Jacket,1017-alyx-9sm,779.0,Coats & Jackets,[],With an unmistakable experimental streetwear a...,"['100% Nylon', 'Stand Up Collar', 'Snap Button...","['https://media.endclothing.com/media/f_auto,q...",The item is an **Oversized Camo Nylon Bomber J...
0,1017 ALYX 9SM Nylon Logo Varsity Jacket,1017-alyx-9sm,489.0,Coats & Jackets,[],"Matthew Williams’s genre-blurring, future-orie...","['100% Polyester', 'Shawl Collar', 'Snap Butto...","['https://media.endclothing.com/media/f_auto,q...",The item is a coat categorized as a varsity ja...
...,...,...,...,...,...,...,...,...,...
0,YMC Embroidered Cap,ymc,62.0,Accessories,[],"You can never have too many caps, so add to yo...","['74% Cotton, 26% Linen', '8-Wale Corduroy', '...","['https://media.endclothing.com/media/f_auto,q...",The YMC Embroidered Cap falls under the catego...
0,YMC Ibiza '89 Pyramid T-Shirt,ymc,47.0,T-Shirts,[],YMC joins forces with Dave Swindells to celebr...,"['100% Organic Slub Cotton', 'Oversized Fit', ...","['https://media.endclothing.com/media/f_auto,q...",The YMC Ibiza '89 Pyramid T-Shirt falls under ...
0,YMC Ibiza '89 Dancers T-Shirt,ymc,43.0,T-Shirts,[],YMC continues to encourage us to create our ow...,"['100% Organic Slub Cotton', 'Oversized Fit', ...","['https://media.endclothing.com/media/f_auto,q...",The YMC Ibiza '89 Dancers T-Shirt is categoriz...
0,YMC Ibiza '89 Sunset T-Shirt,ymc,43.0,T-Shirts,[],Shot over a one week assignment for 20/20 maga...,"['100% Organic Slub Cotton', 'Oversized Fit', ...","['https://media.endclothing.com/media/f_auto,q...",The YMC Ibiza '89 Sunset T-Shirt is categorize...
